In [172]:
import re
import pickle
import pandas as pd
import numpy as np
from numpy import nan

In [173]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [174]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,average_precision_score,recall_score


In [175]:
#PREPROCESSING
def preprocess(col):
    col_l = []
    for r in col:
        #print(r)
        try:
            a = r.lower() 
            a = a.replace("'s", "")
            #a = a.replace(["\n"], "hjfgjh")
            a = re.sub('[^a-zA-Z0-9 \n\.]', '',a)
            a = re.sub('[:.\n\t]', '',a)
            a = re.sub('[?:!.+,]]][]]//;]', '',a) 
            
            col_l.append(a)
        except:
            #print("No header found")
            col_l.append("No Header")
            pass

    # Downloading punkt and wordnet from NLTK
    nltk.download('punkt')
    print("------------------------------------------------------------")
    nltk.download('wordnet')
    
    wordnet_lemmatizer = WordNetLemmatizer()
    
    nltk.download('stopwords')
    
    lemmatized_text_list = []

    for s in col_l:
        #print(s)
        # Create an empty list containing lemmatized words
        lemmatized_list = []

        # Save the text and its words into an object
        #text = df.loc[row]['Content_Parsed_4']
        text_words = s.split(" ")

        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

        # Join the list
        lemmatized_text = " ".join(lemmatized_list)

        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)
    
    new_lemmatized_text_list = []
    for x in lemmatized_text_list:
        new_lemmatized_text_list.append(" ".join(x.split())) 
    
    pl = pd.Series(new_lemmatized_text_list)
   # pl = pd.Series(col_l)
    stop_words = list(stopwords.words('english'))
    
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        #pl = pd.Series(col_l)
   
        pl = pl.str.replace(regex_stopword, '')
        
    
    return pl.tolist()

In [176]:
############Read the new dataset for prediction#########

In [177]:

#read the csv file which containd the data for prediction
dfnew= pd.read_csv('datasetiotnoniot233label.csv')#233 iot and  noniot projects for prediction 
#dfnew= pd.read_csv('datasetiotnoniot233label2.csv')#233  noniot and iot projects for prediction 
#dfnew= pd.read_csv('datasetiotnoniot233label3.csv')#233 noni mixed iot and noniot projects for prediction 
#dfnew= pd.read_csv('dataset75iotnoniotlabel.csv')#75  iot projects for prediction 


In [178]:
cnew=(dfnew['descr']).tolist()
lnew=(dfnew['label']).tolist()

In [179]:
connew=preprocess(cnew)
print(connew)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


------------------------------------------------------------
['   lot  work  image process  need  test cameras imagers   go  create  scalable test platform read    project     lot  work  image process  need  test cameras imagers   go  create  scalable test platform   look  create image process solutions previously  use image sensors  use  parallel interface  cameras  use hdmi interfaces one common interface  image sensors  mipi  let take  look     build  image process design   zybo z7   capable  use  hdmi input   mipi camera interface  also want  lay  grind work   able  create  image process system  allow   quickly  easily test cameras  sensors    future upgrade   project    support  parallel pmod camera  well     able  select   imager  wish  use   algorithm    add advantage   pin    zybo mipi connector   align   commonly use maker cameras  example  pi cameras   look     use  pi camera  another project mipi   commonly use abbreviation   mobile industry processor interface    interface 

In [180]:
df1 = pd.DataFrame({"descr":connew})
df1

,descr
0,lot work image process need test camera...
1,project want share today arduino maze g...
2,pantry automatically replenish kitchen item...
3,make lanethernet relay switch use arduino un...
4,cellular backdoor hologram nova raspberry pi...
...,...
228,ingredients 500g chicken breast 250ml cream 20...
229,homemade bacon truly magical piece meat mak...
230,simple fast delicious im try lowcarb diet w...
231,materials teachers use instructable class...


In [181]:
label=pd.DataFrame({"label":lnew})
label

,label
0,1
1,1
2,1
3,1
4,1
...,...
228,0
229,0
230,0
231,0


In [182]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features =300# not clear how to choose this value

In [183]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

In [184]:
pred_test = tfidf.fit_transform(connew).toarray()
print(pred_test)


[[0.         0.0625754  0.         ... 0.11326679 0.04845431 0.12131186]
 [0.         0.         0.         ... 0.06061332 0.         0.        ]
 [0.06059331 0.06108069 0.         ... 0.         0.0472969  0.11841413]
 ...
 [0.1220065  0.         0.         ... 0.         0.09523376 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.12490196 0.         0.        ]]


In [185]:
################################## DECISION TREE ###############################################

In [186]:
# load the model from disk
loaded_model = pickle.load(open("finalized_model.sav", 'rb'))

DTC_pred1 = loaded_model.predict(pred_test)
print("Decision Tree prediction Accuracy Score  ->",accuracy_score(label, loaded_model.predict(pred_test))*100)
matrix = confusion_matrix(label, loaded_model.predict(pred_test))
print("confusion matrix")
print(matrix)
DTC_pred1

Decision Tree prediction Accuracy Score  -> 99.57081545064378
confusion matrix
[[119   1]
 [  0 113]]


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [187]:
loaded_naive = pickle.load(open("finalized_naive.sav", 'rb'))

Naive_pred1 = loaded_naive.predict(pred_test)
print("Naive Bayes prediction Accuracy Score  ->",accuracy_score(label, loaded_naive.predict(pred_test))*100)

matrix = confusion_matrix(label, loaded_naive.predict(pred_test))
print("confusion matrix")
print(matrix)
Naive_pred1

Naive Bayes prediction Accuracy Score  -> 89.69957081545064
confusion matrix
[[103  17]
 [  7 106]]


array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [188]:
loaded_svm = pickle.load(open("finalized_svm.sav", 'rb'))
print("SVM prediction Accuracy Score  ->",accuracy_score(label, loaded_svm.predict(pred_test))*100)
svm_pred1 = loaded_svm.predict(pred_test)
matrix = confusion_matrix(label, loaded_svm.predict(pred_test))
print("confusion matrix")
print(matrix)
svm_pred1

SVM prediction Accuracy Score  -> 94.4206008583691
confusion matrix
[[111   9]
 [  4 109]]


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [189]:
loaded_knn = pickle.load(open("finalized_knn.sav", 'rb'))
knn_pred1 = loaded_knn.predict(pred_test)
print("KNN prediction Accuracy Score  ->",accuracy_score(label, loaded_knn.predict(pred_test))*100)

matrix = confusion_matrix(label, loaded_knn.predict(pred_test))
print("confusion matrix")
print(matrix)
knn_pred1

KNN prediction Accuracy Score  -> 82.40343347639485
confusion matrix
[[103  17]
 [ 24  89]]


array([1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [190]:
loaded_sgd = pickle.load(open("finalized_sgd.sav", 'rb'))
sgd_pred1 = loaded_sgd.predict(pred_test)
print("SGD prediction Accuracy Score  ->",accuracy_score(label, loaded_sgd.predict(pred_test))*100)

matrix = confusion_matrix(label, loaded_sgd.predict(pred_test))
print("confusion matrix")
print(matrix)
sgd_pred1

SGD prediction Accuracy Score  -> 94.4206008583691
confusion matrix
[[107  13]
 [  0 113]]


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [191]:
loaded_random = pickle.load(open("finalized_random.sav", 'rb'))
random_pred1 = loaded_random.predict(pred_test)
print("Random Forest prediction Accuracy Score  ->",accuracy_score(label, loaded_random.predict(pred_test))*100)

matrix = confusion_matrix(label, loaded_random.predict(pred_test))
print("confusion matrix")
print(matrix)
random_pred1

Random Forest prediction Accuracy Score  -> 95.70815450643777
confusion matrix
[[119   1]
 [  9 104]]


array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)